In [1]:
import numpy

In [2]:
#implementing naive bayes
#create a fit function which creates the dictionary from the training data 
def fit(x_train,y_train):
    result={}
    result["total_data"]=len(y_train)
    class_labels=set(y_train)
    for current_label in class_labels:
        result[current_label]={}
        current_rows=(y_train==current_label)
        x_train_current=x_train[current_rows]
        y_train_current=y_train[current_rows]
        total_words=0
        for i in range(len(f_list)):
            result[current_label][f_list[i]]=x_train_current[:,i].sum()
            total_words+=x_train_current[:,i].sum()
        result[current_label]["total_count"]=total_words
    return result

In [3]:
#function to calculate the probability for a sing message x against class (current_class)
def probability(x,dictionary,current_class):
    output=np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
    for i in range(len(f_list)):
        current_word_count=dictionary[current_class][f_list[i]]+1
        total_word_count=dictionary[current_class]["total_count"]+len(f_list)
        current_word_probability=np.log(current_word_count)-np.log(total_word_count)
        for j in range(int(x[i])): # if the frequency of word in test data point is zero then we wont consider it.
            output+=current_word_probability
    return output

In [4]:
#function to predict the most probable class out of all the classes for a single message x
def predictSingleClass(x,dictionary):
    best_class=-1000
    best_prob=-1000
    firstRun=True
    possible_classes=dictionary.keys()
    for current_class in possible_classes:
        if current_class=="total_data":
            continue
        current_class_probability=probability(x,dictionary,current_class)
        if(firstRun==True or current_class_probability>best_prob):
            best_class=current_class
            best_prob=current_class_probability
        firstRun=False
    return best_class

In [5]:
#this function predicts the output for the complete testing data x_test
def predict(X_test,dictionary):
    Y_pred=[]
    num = 0
    for x in X_test:
        Y_pred.append(predictSingleClass(x,dictionary))
    return Y_pred

In [ ]:
#tarfile helps to extract the tar.gz file into the given directory 
import sys
import os
import urllib.request
import tarfile
import zipfile
tarfile.open('C:/Users/GANDHARAV/Downloads/20_newsgroups.tar.gz').extractall('data2/20_newsgroups')

In [6]:
from sklearn import datasets 

In [7]:
#loading all the newsgroupfolders using the load_files function and create an obj that can be used to extract the data
obj=datasets.load_files(r'C:/Users/GANDHARAV/datascience and ml programs/data2/20_newsgroups/20_newsgroups')

In [16]:
obj.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [8]:
#extract all the data of the newsgroup messages form the data attribute of the object and create a numpy array
messages=obj.data
import numpy as np
arr=np.array(messages)
type(arr)

numpy.ndarray

In [9]:
#extarct the stopwords and the punctuations form the relevant module.stopwords and punctuations will not be included in the features
import nltk
from nltk.corpus import stopwords
from string import punctuation
pun=list(punctuation)
sw=stopwords.words('english')
sw+=pun

In [15]:
#the numpy array (arr) contains all the messages in the numpy.bytes_ format, these messages are required to be converted to string format
#so that we can detemine the features
#also create a dictionary which will store the words in all the texts as keys and their corresponding frequency as values
tb=nltk.tokenize.TreebankWordTokenizer()
data_arr=[]
for i in range(len(arr)):
    data_arr.append(arr[i].decode('utf-8','ignore'))
di={}
for row in data_arr:
    li=tb.tokenize(row)
    for x in li:
        if x.lower() not in sw and len(x.lower())>=5:
            if x.lower() not in di:
                di[x.lower()]=1
            else:
                di[x.lower()]+=1
len(di)

270349

In [17]:
#arrange the dictionary in descending order of frequency of the words and include the top 250 words as the feature set
feature_list=sorted(di.items(),key=lambda k:(k[1],k[0]),reverse=True)
f_list=[]
for i in feature_list:
    f_list.append(i[0])
f_list=f_list[:250]

In [14]:
#also include these words with the stopwords ,so that these words are also not included within the feature set
sw+=['subject','from', 'date', 'newsgroups', 'message-id', 'lines', 'path', 'organization','please','thanks','rather','called','though','given','makes', 
            'would', 'writes', 'references', 'article', 'sender', 'nntp-posting-host', 'people', 
            'university', 'think', 'xref', 'cantaloupe.srv.cs.cmu.edu', 'could', 'distribution', 'first', 
            'anyone','world', 'really', 'since', 'right', 'believe', 'still', 
            "max>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'ax>'"]

In [20]:
#create a dataframe which consists the features as its columns. each row in this dataframe will corresond to a different message 
# and each of the column contains the number of times the word has appeared in the message
import pandas as pd
X=pd.DataFrame(columns=f_list)
for row in data_arr:
    X.loc[len(X)]=np.zeros(len(f_list))
    li=tb.tokenize(row)
    for j in li:
        if j.lower() in f_list:
            X[j.lower()][len(X)-1]+=1


In [22]:
#split the dataset into training and testing dataset and predict the results using the naive bayes functions
#also print the classification_report and confusion matrix for accuracy measurements
# the accuracy of the results is nearly 91%
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
x_arr=np.array(X)
x_train,x_test,y_train,y_test=train_test_split(x_arr,obj.target,random_state=0)
dictionary=fit(x_train,y_train)
y_pred=predict(x_test,dictionary)
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       285
           1       0.93      0.93      0.93       243
           2       0.97      0.95      0.96       242
           3       0.90      0.95      0.92       242
           4       0.96      0.96      0.96       265
           5       0.95      0.97      0.96       259
           6       0.96      0.89      0.93       246
           7       0.95      0.96      0.95       226
           8       0.98      0.98      0.98       262
           9       0.98      0.96      0.97       247
          10       0.97      0.97      0.97       262
          11       0.95      0.94      0.95       246
          12       0.96      0.97      0.97       247
          13       0.97      0.97      0.97       231
          14       0.94      0.97      0.96       241
          15       0.99      0.98      0.98       252
          16       0.93      0.83      0.88       290
          17       0.91    

In [27]:
#prediction of the classes in terms of actual classes 
l=list(obj.target_names)
d1={}
ans_l=[]
q=0
for i in l:
    d1[q]=i
    q+=1
for j in y_pred:
    ans_l.append(d1[j])
ans_l

['alt.atheism',
 'sci.electronics',
 'comp.graphics',
 'talk.religion.misc',
 'rec.sport.hockey',
 'misc.forsale',
 'talk.politics.misc',
 'rec.sport.hockey',
 'alt.atheism',
 'soc.religion.christian',
 'comp.graphics',
 'sci.electronics',
 'alt.atheism',
 'rec.sport.hockey',
 'rec.autos',
 'talk.politics.misc',
 'rec.motorcycles',
 'comp.sys.ibm.pc.hardware',
 'talk.politics.misc',
 'comp.windows.x',
 'rec.motorcycles',
 'rec.sport.hockey',
 'talk.politics.misc',
 'comp.sys.ibm.pc.hardware',
 'talk.politics.guns',
 'rec.motorcycles',
 'rec.autos',
 'comp.sys.mac.hardware',
 'soc.religion.christian',
 'soc.religion.christian',
 'sci.space',
 'talk.religion.misc',
 'alt.atheism',
 'comp.os.ms-windows.misc',
 'talk.politics.guns',
 'rec.sport.baseball',
 'alt.atheism',
 'misc.forsale',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.politics.mideast',
 'rec.motorcycles',
 'comp.windows.x',
 'rec.motorcycles',
 'alt.atheism',
 'comp.sys.mac.hardware',
 'comp.os.ms-windows.misc',
 

In [25]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_pred,y_test))

[[204   0   1   0   0   1   0   0   1   0   1   0   0   0   2   1   0   1
    0  73]
 [  0 226   0   5   0   3   0   0   0   0   0   1   0   2   6   0   0   0
    0   0]
 [  0   2 229   5   0   4   0   1   0   0   0   0   1   0   0   0   0   0
    0   0]
 [  0   4   4 230   1   0   2   0   0   0   0   0   0   0   0   0   0   0
    1   0]
 [  0   1   0   4 254   0   1   0   0   0   0   1   2   0   1   0   0   0
    1   0]
 [  0   2   2   1   0 250   1   0   0   0   0   2   0   0   0   0   0   0
    1   0]
 [  0   1   0   3   3   3 220   2   0   0   2   1   3   2   1   0   3   1
    1   0]
 [  1   1   1   1   1   0   2 216   0   0   1   0   1   0   0   0   0   0
    1   0]
 [  0   1   0   0   1   0   0   1 256   0   0   0   0   0   0   0   0   3
    0   0]
 [  2   1   0   0   1   0   0   0   1 237   2   1   0   0   1   0   0   0
    1   0]
 [  1   0   0   1   0   1   1   0   1   1 253   0   1   0   1   0   1   0
    0   0]
 [  0   0   0   0   1   1   1   4   1   1   0 232   0   0   0   0

In [29]:
#predicting the results from inbulit naive bayes model and getting the coresponding accuracy score
#storing the results from inbuilt score in the variable y_inbuilt_pred
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()
mnb.fit(x_train,y_train)
y_inbuilt_pred=mnb.predict(x_test)

In [31]:
#printing the classification_report using the prediction results from the inbuilt naive bayes algorithm and the  one which
#was implemented previously so that we can compare the results form both the implementations....and the sccuracy is =99%...
# which shows that the results from both the implementations are almost equal
print(classification_report(y_pred,y_inbuilt_pred))
print(confusion_matrix(y_pred,y_inbuilt_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       285
           1       1.00      0.99      0.99       243
           2       1.00      1.00      1.00       242
           3       1.00      1.00      1.00       242
           4       1.00      1.00      1.00       265
           5       1.00      1.00      1.00       259
           6       0.97      1.00      0.98       246
           7       0.99      1.00      1.00       226
           8       0.99      1.00      0.99       262
           9       0.98      1.00      0.99       247
          10       0.99      1.00      0.99       262
          11       1.00      0.99      0.99       246
          12       0.99      1.00      1.00       247
          13       0.99      1.00      0.99       231
          14       1.00      0.99      0.99       241
          15       1.00      1.00      1.00       252
          16       0.98      0.99      0.98       290
          17       1.00    